In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Subject_name = ["001"]
Session_name = ["11", "12"]
Block_name = ["1", "2", "3", "4"]

TR_threshold = 3
Block_duration = 5 * 60 # sec
TR_tolerance = 5 # sec
Shock_duration = 0.05 # sec
Shock_delay = 0.5 # sec
Visual_shock_duration = 2 # sec

Size_room = (18, 18)
Size_grand = (78, 78)
eplison = 1e-4
room_range = (0.5 + eplison, 0.5 + eplison) # per size_room

# Pre: -2?, -1?; Room: -1?, 0; Onset: 0, 0.05; Post: 0.05, 1?
Event_folder = "Events/"
Onset_phases = ["Onset", "OnsetPre", "OnsetRoom", "OnsetPost"]
NonOnset_phases = ["Pre", "Room", "Post"]
Func_scan_stim_range = [[0, Shock_duration],
                        [-1, 0],
                        [0, 0],
                        [0, 1],
                        [-1, 0],
                        [0, 0],
                        [Shock_delay, Shock_delay + 1]] # sec
if len(Onset_phases) + len(NonOnset_phases) != len(Func_scan_stim_range):
    print("Unequal length of phase and stim range!")
    

In [2]:
def search_time_index(table, time, l, r, label = "timestamp"):
    while r - l > 1:
        m = int((l + r) / 2.0)
        if time < table[label].iloc[l]:
            return l
        if time > table[label].iloc[r]:
            return r
        if time < table[label].iloc[m]:
            r = m - 1
        else:
            l = m
    return l
    

In [3]:
Stim_table = pd.read_csv("PainSignalRecord.csv")
Stim_table = Stim_table.drop(labels = Stim_table[(Stim_table.timestamp == "timestamp")].index).reset_index(drop = True)
Stim_table[["timestamp", "bodyPart"]] = Stim_table[["timestamp", "bodyPart"]].astype(float)
print('Stimuli timetable')
print(Stim_table)

TR_table = pd.read_csv("TRSignalRecord.csv")
TR_table = TR_table.drop(labels = TR_table[(TR_table.timestamp == "timestamp")].index).reset_index(drop = True)
TR_table[TR_table.columns] = TR_table[TR_table.columns].astype(float)
print('TR timetable')
print(TR_table)


Stimuli timetable
        timestamp  bodyPart mode
0    1.674657e+09       0.0    R
1    1.674657e+09       0.0    W
2    1.674657e+09       0.0    R
3    1.674657e+09       0.0    W
4    1.674657e+09       1.0    R
..            ...       ...  ...
425  1.674664e+09       1.0    W
426  1.674664e+09       1.0    R
427  1.674664e+09       1.0    W
428  1.674664e+09       0.0    R
429  1.674664e+09       0.0    W

[430 rows x 3 columns]


C:\Users\chron\anaconda3\envs\pytorch\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


TR timetable
           timestamp         TR
0       1.674656e+09  -0.000436
1       1.674656e+09  -0.000436
2       1.674656e+09  -0.000436
3       1.674656e+09  -0.001093
4       1.674656e+09  -0.000107
...              ...        ...
640102  1.674664e+09 -10.653455
640103  1.674664e+09 -10.653455
640104  1.674664e+09 -10.653455
640105  1.674664e+09 -10.653455
640106  1.674664e+09 -10.653455

[640107 rows x 2 columns]


In [4]:
def is_in_room(table, index, i_range = [-1, 0, 1]):
    curr = (table["playerPositionX"].iloc[index], table["playerPositionZ"].iloc[index])
    for i in i_range:
        for j in i_range:
            if (abs(curr[0] - i * Size_grand[0]) < room_range[0] * Size_room[0]) and (abs(curr[1] - j * Size_grand[1]) < room_range[1] * Size_room[1]):
                return True
    return False
    

In [5]:
def Event_process(subj, sess, blk, Stim, TR, blk_start_time):
    shock_TR, shock_game, diff_exit_onset = [], [], []
    temporal_correction = 0
    
    file_game_csv = "Data/Sub" + subj + "_S" + sess + "_T" + blk + ".csv"
    
    stim_start_index = search_time_index(Stim, blk_start_time, 0, len(Stim) - 1)
    while (Stim["timestamp"].iloc[stim_start_index] < blk_start_time) or (Stim["mode"].iloc[stim_start_index] != "W"):
        stim_start_index += 1
    stim_index = stim_start_index
    
    scan_start_index = search_time_index(TR, Stim["timestamp"].iloc[stim_index], 0, len(TR) - 1)
    prev_time, prev_index = TR["timestamp"].iloc[scan_start_index], scan_start_index
    while scan_start_index:
        scan_start_index -= 1
        if TR["TR"].iloc[scan_start_index] <= TR_threshold:
            if prev_time - TR["timestamp"].iloc[scan_start_index] > TR_tolerance:
                scan_start_index = prev_index
                break
        else:
            prev_time, prev_index = TR["timestamp"].iloc[scan_start_index], scan_start_index
    
    while stim_index < len(Stim):
        if Stim["mode"].iloc[stim_index] != "W":
            stim_index += 1
            continue
#         if stim_index < len(Stim) - 2: # exclude the former shock if two shocks within 2 sec
#             if Stim["timestamp"].iloc[stim_index + 2] - Stim["timestamp"].iloc[stim_index] < Visual_shock_duration:
#                 stim_index += 1
#                 continue
        scan_index = search_time_index(TR, Stim["timestamp"].iloc[stim_index], 0, len(TR) - 1)

        Game_table = pd.read_csv(file_game_csv)
        game_start_index = search_time_index(Game_table, TR["timestamp"].iloc[scan_index] - TR["timestamp"].iloc[scan_start_index], 0, len(Game_table) - 1, label = "time")
        game_index = game_start_index
        while game_index < len(Game_table) - 1:
            if int(Game_table["painBodyPart"].iloc[game_index]):
                # Find the last recorded shocks for 2 sec if the whole segment is above 2 sec 
#                 game_shock_end_index = search_time_index(Game_table, Game_table["time"].iloc[game_index] + Visual_shock_duration, 0, len(Game_table) - 1, label = "time")
#                 if not int(Game_table["painBodyPart"].iloc[game_shock_end_index + 1]):
                shock_game.append(round(Game_table["time"].iloc[game_index], 2))
                break
            game_index += 1

        if temporal_correction == 0:
            game_i = search_time_index(Game_table, shock_game[0], 0, len(Game_table) - 1, label = "time")
            for i in range(game_i):
                if int(Game_table["painBodyPart"].iloc[i]):
                    temporal_correction = shock_game[0] - Game_table["time"].iloc[i]
                    break
        
        for ph in range(len(Onset_phases)):
            phase, stim_range = Onset_phases[ph] , Func_scan_stim_range[ph]
            try:
                os.mkdir(os.path.join(Event_folder, phase))
            except OSError as error: 
                pass
            file_name = Event_folder + phase + "/S" + sess + "_T" + blk + ".txt"
            
            room_temporal_range = [0, 0]
            game_room_index = game_index
            while not is_in_room(Game_table, game_room_index):
                game_room_index -= 1
            room_temporal_range[1] = Game_table["time"].iloc[game_room_index] - Game_table["time"].iloc[game_index]
            while is_in_room(Game_table, game_room_index):
                game_room_index -= 1
            room_temporal_range[0] = Game_table["time"].iloc[game_room_index] - Game_table["time"].iloc[game_index]

            scan_stim_time = TR["timestamp"].iloc[scan_index] + stim_range[0] - TR["timestamp"].iloc[scan_start_index]
            scan_stim_duration = stim_range[1] - stim_range[0]
            if (scan_stim_time > Block_duration) or (scan_stim_time < 0):
                break
            if (phase == "OnsetPre") or (phase == "OnsetRoom"):
                scan_stim_time += room_temporal_range[0]
                if phase == "OnsetRoom":
                    scan_stim_duration += (room_temporal_range[1] - room_temporal_range[0])
            target_start_time = round(scan_stim_time, 2)
            target_stim_duration = round(scan_stim_duration, 2)
            if phase == "Onset":
                diff_exit_onset.append(round(0 - room_temporal_range[1], 2))
                shock_TR.append(target_start_time)
            with open(file_name, 'a+') as f:
                f.write(str(target_start_time) + ' ' + str(target_stim_duration) + ' 1')
                f.write('\n')

        stim_index += 1
    
    
    print("======================Onset check======================")
    print("S" + sess + "_T" + blk)
    print("First TR  :" + str(TR["timestamp"].iloc[scan_start_index]))
    print("First Stim:" + str(Stim["timestamp"].iloc[stim_start_index]))
    print("Delta-T:" + str(Stim["timestamp"].iloc[stim_start_index] - TR["timestamp"].iloc[scan_start_index]))
    if temporal_correction > Visual_shock_duration:
        print("Unalignment with the first recorded shock!")
    elif temporal_correction == 0:
        print("Unfinished temporal correction!")
    else:
        print("Temporal correction (sec):", temporal_correction)
#         print("Time between room exit and shock onset:", [min(diff_exit_onset), max(diff_exit_onset)])
    print("Time between room exit and shock onset:", diff_exit_onset)
    print("Shock time from python:", shock_TR)
    print("Shock time from game:", shock_game)
    if len(shock_TR) != len(shock_game):
        print("Unalignment number of recorded shocks!")
        print(shock_TR)
        print(shock_game)
    else:
        print("Max diff between recorded shocks (sec):", round(np.max(np.abs(np.array(shock_TR) - np.array(shock_game))), 2))
        unrec_shock_time = []
        game_i = search_time_index(Game_table, shock_game[0] - Visual_shock_duration, 0, len(Game_table) - 1, label = "time")
        game_j = search_time_index(Game_table, shock_game[-1] + Visual_shock_duration, 0, len(Game_table) - 1, label = "time")
        for i in range(game_i):
            if int(Game_table["painBodyPart"].iloc[i]):
                unrec_shock_time.append(Game_table["time"].iloc[i])
        for j in range(game_j + 1, len(Game_table) - 1):
            if int(Game_table["painBodyPart"].iloc[j]):
                unrec_shock_time.append(Game_table["time"].iloc[j])
        print("Unrecorded shocks:", unrec_shock_time)
    
    
    game_index = 0
    is_room = True
    is_start = True
    room_temporal_range = [0, 0]
    while game_index < len(Game_table):
        if is_start and not is_in_room(Game_table, game_index):
            is_room = False
            is_start = False
        if is_start:
            game_index += 1
            continue
        if not is_room and is_in_room(Game_table, game_index):
            is_room = True
            room_temporal_range[0] = Game_table["time"].iloc[game_index]
            game_index += 1
            continue
        if is_room and (not is_in_room(Game_table, game_index) or (game_index == len(Game_table)- 1)):
            is_room = False
            room_temporal_range[1] = Game_table["time"].iloc[game_index]
            for ph in range(len(NonOnset_phases)):
                phase, stim_range = NonOnset_phases[ph] , Func_scan_stim_range[ph + len(Onset_phases)]
                try:
                    os.mkdir(os.path.join(Event_folder, phase))
                except OSError as error: 
                    pass
                file_name = Event_folder + phase + "/S" + sess + "_T" + blk + ".txt"
                
                scan_stim_time = room_temporal_range[0] + stim_range[0]
                scan_stim_duration = stim_range[1] - stim_range[0]
                if phase == "Room":
                    scan_stim_duration += (room_temporal_range[1] - room_temporal_range[0])
                target_start_time = round(scan_stim_time, 2)
                target_stim_duration = round(scan_stim_duration, 2)
                with open(file_name, 'a+') as f:
                    f.write(str(target_start_time) + ' ' + str(target_stim_duration) + ' 1')
                    f.write('\n')
        game_index += 1
        


In [6]:
def Event_file_creation():
    for subj in Subject_name:
        for sess in Session_name:
            for blk in Block_name:
                file_name = "Info/Sub" + subj + "_S" + sess + "_T" + blk + ".txt"
                for line in open(file_name):
                    if "trialStartTime" in line:
                        trial_start_time = float(line[16:])
                        Event_process(subj, sess, blk, Stim_table, TR_table, trial_start_time)
                        break


In [7]:
Event_file_creation()
    

======================Onset check======================
S11_T1
First TR  :1674660964.6715274
First Stim:1674660975.089805
Delta-T:10.418277502059937
Temporal correction (sec): 0.4565549999999998
Time between room exit and shock onset: [0.52, 0.54, 0.52, 0.51, 0.53, 0.51, 0.53, 0.5, 0.5, 0.55, 0.52, 0.54, 0.51, 0.53, 0.51, 0.52, 0.52, 0.52, 0.52, 0.5, 0.52, 0.55, 0.51, 0.52, 0.52]
Shock time from python: [10.41, 43.87, 54.06, 59.56, 68.15, 73.43, 78.39, 98.38, 107.22, 112.41, 122.41, 136.94, 141.74, 146.54, 151.45, 156.36, 161.41, 166.3, 176.98, 182.45, 187.36, 192.36, 197.59, 207.2, 219.94]
Shock time from game: [10.41, 43.87, 54.06, 59.56, 68.13, 73.43, 78.39, 98.37, 107.21, 112.4, 122.41, 136.94, 141.74, 146.54, 151.44, 156.36, 161.41, 166.3, 176.97, 182.44, 187.36, 192.36, 197.59, 207.2, 219.94]
Max diff between recorded shocks (sec): 0.02
Unrecorded shocks: []
======================Onset check======================
S11_T2
First TR  :1674661293.1621382
First Stim:1674661321.3248832
